In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.io import loadmat


In [69]:
def load_data(filepath):
    """
    Load Burgers' equation data or similar PDE dataset.
    """
    data = loadmat(filepath)
    t = np.real(data['t'].flatten())[:, None]  # Time points
    x = np.real(data['x'].flatten())[:, None]  # Spatial points
    Exact = np.real(data['usol']).T           # Solution matrix

    # Create space-time grid
    X, T = np.meshgrid(x, t)
    X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    u_star = Exact.flatten()[:, None]

    # Bounds for normalization
    lb = X_star.min(0)
    ub = X_star.max(0)

    return X_star, u_star, lb, ub, Exact, X, T


In [70]:
def prepare_data(X_star, u_star, collocation_points, noise=0.01, N_u_s=100, split=0.8):
    """
    Prepare measurement data and collocation points.
    """
    # Randomly sample measurement points
    idx_s = np.random.choice(X_star.shape[0], N_u_s, replace=False)
    X_u_meas = X_star[idx_s, :]
    u_meas = u_star[idx_s, :]

    # Add noise to measurements
    u_meas += noise * np.std(u_meas) * np.random.randn(*u_meas.shape)

    # Split measurement data into training and validation
    N_u_train = int(split * N_u_s)
    idx_train = np.random.choice(N_u_s, N_u_train, replace=False)

    X_u_train = X_u_meas[idx_train, :]
    u_train = u_meas[idx_train, :]

    idx_val = np.setdiff1d(np.arange(N_u_s), idx_train)
    X_u_val = X_u_meas[idx_val, :]
    u_val = u_meas[idx_val, :]

    # Use provided collocation points
    X_f_train = collocation_points

    return X_u_train, u_train, X_u_val, u_val, X_f_train


In [71]:
class PhysicsInformedNN(nn.Module):
    def __init__(self, X, u, X_f, X_val, u_val, layers, lb, ub):
        super(PhysicsInformedNN, self).__init__()
        self.lb = torch.tensor(lb, dtype=torch.float32)
        self.ub = torch.tensor(ub, dtype=torch.float32)
        self.model = self._initialize_nn(layers)

        self.lambda1 = nn.Parameter(torch.zeros(16, 1, dtype=torch.float32), requires_grad=True)

        self.x = torch.tensor(X[:, 0:1], dtype=torch.float32)
        self.t = torch.tensor(X[:, 1:2], dtype=torch.float32)
        self.u = torch.tensor(u, dtype=torch.float32)
        self.x_f = torch.tensor(X_f[:, 0:1], dtype=torch.float32)
        self.t_f = torch.tensor(X_f[:, 1:2], dtype=torch.float32)

        self.x_val = torch.tensor(X_val[:, 0:1], dtype=torch.float32)
        self.t_val = torch.tensor(X_val[:, 1:2], dtype=torch.float32)
        self.u_val = torch.tensor(u_val, dtype=torch.float32)

    def _initialize_nn(self, layers):
        modules = []
        for i in range(len(layers) - 1):
            modules.append(nn.Linear(layers[i], layers[i + 1]))
            if i < len(layers) - 2:
                modules.append(nn.Tanh())

        model = nn.Sequential(*modules)

        for layer in model:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

        return model

    def forward(self, x, t):
        inputs = torch.cat([x, t], dim=1)
        inputs = 2.0 * (inputs - self.lb) / (self.ub - self.lb) - 1.0
        return self.model(inputs)

    def net_u(self, x, t):
        return self.forward(x, t)

    def net_f(self, x, t):
        u = self.net_u(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]

        Phi = torch.cat([torch.ones_like(u), u, u_x, u_xx, u ** 2], dim=1)

        if self.training:
            self.lambda1.data = STRidge(u_t, Phi, delta=1e-3, max_iter=100)

        f = u_t + Phi @ self.lambda1
        return f, Phi, u_t

    def compute_loss(self):
        u_pred = self.net_u(self.x, self.t)
        loss_u = torch.mean((self.u - u_pred) ** 2)

        f_pred, Phi_pred, u_t_pred = self.net_f(self.x_f, self.t_f)
        loss_f = torch.mean(f_pred ** 2)

        loss_lambda = 1e-7 * torch.norm(self.lambda1, p=1)

        return torch.log(loss_u + loss_f + loss_lambda)


In [72]:
def STRidge(U_dot, Phi, delta, max_iter=1000, ridge_param=1e-5):
    M = Phi.shape[1]
    lambda_ = torch.linalg.solve(Phi.T @ Phi + ridge_param * torch.eye(M), Phi.T @ U_dot)

    for _ in range(max_iter):
        small_idx = torch.abs(lambda_) < delta
        large_idx = ~small_idx

        lambda_[small_idx] = 0.0

        if torch.any(large_idx):
            Phi_large = Phi[:, large_idx]
            lambda_large = torch.linalg.solve(
                Phi_large.T @ Phi_large + ridge_param * torch.eye(Phi_large.shape[1]),
                Phi_large.T @ U_dot
            )
            lambda_[large_idx] = lambda_large

    return lambda_


In [73]:
def train_with_ADO(model, optimizers, epochs, X_u_train, u_train, X_f_train):
    history = []
    adam_optimizer = optimizers["adam"]

    for epoch in range(epochs):
        def closure():
            adam_optimizer.zero_grad()
            loss = model.compute_loss()
            loss.backward()
            return loss

        adam_optimizer.step(closure)

        with torch.no_grad():
            f_pred, Phi, u_t_pred = model.net_f(model.x_f, model.t_f)
            model.lambda1.data = STRidge(u_t_pred, Phi, delta=1e-3, max_iter=100)

        with torch.no_grad():
            loss = model.compute_loss()
            history.append(loss.item())

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs}: Loss={loss.item()}")

    return history


In [74]:
def extract_pde(lambda1, terms):
    lambda_values = lambda1.detach().numpy().flatten()
    discovered_terms = [
        f"{coef:.6f}*{term}" for coef, term in zip(lambda_values, terms) if abs(coef) > 1e-5
    ]
    return " + ".join(discovered_terms)


In [75]:
# Load and prepare data
filepath = "data/burgers.mat"
X_star, u_star, lb, ub, Exact, X, T = load_data(filepath)
collocation_points = np.random.rand(1000, 2)
X_u_train, u_train, X_u_val, u_val, X_f_train = prepare_data(X_star, u_star, collocation_points)

# Initialize model
layers = [2, 50, 50, 1]
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, X_u_val, u_val, layers, lb, ub)

# Initialize optimizers
optimizers = {"adam": optim.Adam(model.parameters(), lr=1e-3)}

# Train model with ADO and STRidge
train_with_ADO(model, optimizers, epochs=1000, X_u_train=X_u_train, u_train=u_train, X_f_train=X_f_train)

# Extract discovered PDE
terms = ["1", "u", "u_x", "u_xx", "u_t"]
discovered_pde = extract_pde(model.lambda1, terms)
print("Discovered PDE:", discovered_pde)


RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
import numpy as np
import tensorflow as tf

# Generate synthetic data
np.random.seed(1234)
X_u_train = np.random.rand(100, 2)  # Input data (x, t)
u_train = np.sin(X_u_train[:, 0:1]) * np.cos(X_u_train[:, 1:2])  # Target data

X_f_train = np.random.rand(200, 2)  # Collocation points for physics constraints
X_u_val = np.random.rand(50, 2)  # Validation points
u_val = np.sin(X_u_val[:, 0:1]) * np.cos(X_u_val[:, 1:2])  # Validation data

lb = np.array([0, 0])  # Lower bounds of the domain
ub = np.array([1, 1])  # Upper bounds of the domain


In [ ]:
import tensorflow as tf
import numpy as np

class PhysicsInformedNN:
    def __init__(self, X, u, X_f, layers, lb, ub):
        """
        Initialize the Physics-Informed Neural Network.

        Args:
            X (np.ndarray): Training inputs (space and time).
            u (np.ndarray): Training outputs.
            X_f (np.ndarray): Collocation points for physics-based constraints.
            layers (list): Network architecture.
            lb (np.ndarray): Lower bound of the domain.
            ub (np.ndarray): Upper bound of the domain.
        """
        self.lb = tf.constant(lb, dtype=tf.float32)
        self.ub = tf.constant(ub, dtype=tf.float32)
        self.layers = layers

        # Build the neural network
        self.model = self.build_model(layers)
        self.lambda1 = tf.Variable(tf.zeros([16, 1], dtype=tf.float32), name="lambda")

        # Store data
        self.X = tf.convert_to_tensor(X, dtype=tf.float32)  # Ensure input data is converted to tensors
        self.u = tf.convert_to_tensor(u, dtype=tf.float32)  # Convert to Tensor
        self.X_f = tf.convert_to_tensor(X_f, dtype=tf.float32)  # Collocation points

        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        self.loss_history = []

    def build_model(self, layers):
        """
        Build a feedforward neural network using Keras layers.
        """
        model = tf.keras.Sequential()
        for i in range(len(layers) - 1):
            model.add(tf.keras.layers.Dense(
                layers[i + 1],
                activation='tanh' if i < len(layers) - 2 else None,
                kernel_initializer='glorot_normal'
            ))
        return model

    def net_u(self, x, t):
        """
        Compute the neural network output (u) based on spatial and temporal inputs.
        """
        X_scaled = 2.0 * (tf.concat([x, t], axis=1) - self.lb) / (self.ub - self.lb) - 1.0
        return self.model(X_scaled)

    def net_f(self, x, t):
        """
        Compute the physics-based loss function (f) based on the network output.
        """
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch([x, t])
            u = self.net_u(x, t)

            # First-order derivatives
            u_t = tape1.gradient(u, t)
            u_x = tape1.gradient(u, x)

        with tf.GradientTape() as tape2:
            tape2.watch(x)
            u_xx = tape2.gradient(u_x, x)

        # Example PDE: u_t - λ₁ * u - λ₂ * u_xx = 0
        f = u_t - self.lambda1[0] * u - self.lambda1[1] * u_xx
        return f

    def compute_loss(self):
        """
        Compute the total loss.
        """
        # Data loss
        with tf.GradientTape() as tape:
            u_pred = self.net_u(self.X[:, 0:1], self.X[:, 1:2])
        loss_u = tf.reduce_mean(tf.square(self.u - u_pred))

        # Physics loss
        f_pred = self.net_f(self.X_f[:, 0:1], self.X_f[:, 1:2])
        loss_f = tf.reduce_mean(tf.square(f_pred))

        # Regularization term for lambda
        loss_lambda = 1e-7 * tf.reduce_sum(tf.abs(self.lambda1))

        return loss_u + loss_f + loss_lambda

    def train(self, epochs):
        """
        Train the PINN using a custom training loop.
        """
        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                loss = self.compute_loss()

            # Compute gradients
            grads = tape.gradient(loss, self.model.trainable_variables + [self.lambda1])

            # Apply gradients
            self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables + [self.lambda1]))

            self.loss_history.append(loss.numpy())

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss.numpy()}")



In [ ]:
def STRidge(U, Phi, delta, max_iter=10):
    """
    Perform Sparse Threshold Ridge Regression.

    Args:
        U (np.ndarray): Target vector (e.g., time derivatives).
        Phi (np.ndarray): Candidate function library.
        delta (float): Threshold for sparsity.
        max_iter (int): Maximum iterations.

    Returns:
        np.ndarray: Coefficients of the sparse solution.
    """
    Lambda = np.linalg.lstsq(Phi, U, rcond=None)[0]  # Initial Ridge Regression
    for _ in range(max_iter):
        small_idx = np.abs(Lambda) < delta
        Lambda[small_idx] = 0
        large_idx = ~small_idx
        if np.sum(large_idx) == 0:
            break
        Phi_reduced = Phi[:, large_idx]
        Lambda[large_idx] = np.linalg.lstsq(Phi_reduced, U, rcond=None)[0]
    return Lambda


In [ ]:
def ADO(pinn, tol, delta, n_iter):
    """
    Alternating Direction Optimization (ADO) framework.

    Args:
        pinn (PhysicsInformedNN): The PINN model instance.
        tol (float): Convergence tolerance.
        delta (float): Threshold for STRidge sparsity.
        n_iter (int): Number of iterations.
    """
    for iteration in range(n_iter):
        # Train the PINN
        pinn.train(epochs=100)

        # Extract candidate terms
        X_f = pinn.X_f.numpy()
        derivatives = pinn.compute_derivatives(tf.convert_to_tensor(X_f))
        Phi = np.column_stack([derivatives["u"], derivatives["u_x"], derivatives["u_xx"]])
        U = derivatives["u_t"].numpy()

        # Apply STRidge
        Lambda = STRidge(U, Phi, delta)
        print(f"Iteration {iteration}, Lambda: {Lambda}")


In [ ]:
# Example data
X_u_train = np.random.rand(100, 2)  # Spatial and temporal coordinates
u_train = np.sin(X_u_train[:, 0]) * np.cos(X_u_train[:, 1])  # Example u
X_f_train = np.random.rand(100, 2)  # Collocation points
layers = [2, 20, 20, 1]  # Example network layers
lb = np.array([0, 0])  # Lower bound of the domain
ub = np.array([1, 1])  # Upper bound of the domain

# Initialize and train the model
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub)
model.train(epochs=1000)


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:
# Define the PINN model
layers = [2, 64, 64, 1]
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub)

# Use ADO and STRidge for PDE discovery
ADO(model, tol=1e-3, delta=1e-4, n_iter=10)


TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from scipy.spatial import distance
from matplotlib import cm
import time
from mpl_toolkits.mplot3d import Axes3D
from pyDOE import lhs  # Latin Hypercube Sampling for collocation points
import sobol_seq  # Sobol sequence generator for collocation points
import os

# Set device to CPU explicitly
with tf.device('/CPU:0'):
    pass  # Replace with computations later if needed


In [ ]:
# =============================================================================
#  Define loss histories to record convergence
# =============================================================================

# Adam loss history
loss_history_Adam = np.array([0])
loss_u_history_Adam = np.array([0])
loss_f_history_Adam = np.array([0])
loss_lambda_history_Adam = np.array([0])
lambda_history_Adam = np.zeros((16, 1))
loss_history_Adam_val = np.array([0])
loss_u_history_Adam_val = np.array([0])
loss_f_history_Adam_val = np.array([0])

# STRidge loss history
loss_history_STRidge = np.array([0])
loss_f_history_STRidge = np.array([0])
loss_lambda_history_STRidge = np.array([0])
optimaltol_history = np.array([0])
tol_history_STRidge = np.array([0])
lambda_normalized_history_STRidge = np.zeros((16, 1))

lambda_history_STRidge = np.zeros((16, 1))
ridge_append_counter_STRidge = np.array([0])

# Loss histories for pretraining
loss_history_Pretrain = np.array([0])
loss_u_history_Pretrain = np.array([0])
loss_f_history_Pretrain = np.array([0])
loss_lambda_history_Pretrain = np.array([0])
loss_history_val_Pretrain = np.array([0])
loss_u_history_val_Pretrain = np.array([0])
loss_f_history_val_Pretrain = np.array([0])
step_Pretrain = 0

lambda_history_Pretrain = np.zeros((16, 1))

# Set random seeds for reproducibility
np.random.seed(1234)
tf.random.set_seed(1234)  # Updated for TensorFlow 2.x


In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
import os
from scipy.interpolate import griddata
from scipy.spatial import distance
import time
from mpl_toolkits.mplot3d import Axes3D
from pyDOE import lhs

class PhysicsInformedNN:
    def __init__(self, X, u, X_f, X_val, u_val, layers, lb, ub):
        # Store domain bounds
        self.lb = lb
        self.ub = ub
        self.layers = layers
        
        # Initialize weights and biases
        self.weights, self.biases = self.initialize_NN(layers)

        # Eager execution
        tf.compat.v1.enable_eager_execution()

        # Initialize model parameters
        self.lambda1 = tf.Variable(tf.zeros([16, 1], dtype=tf.float32), dtype=tf.float32, name='lambda')


        # Training data
        self.x = X[:, 0:1]
        self.t = X[:, 1:2]
        self.u = u
        self.x_f = X_f[:, 0:1]
        self.t_f = X_f[:, 1:2]

        # Use TensorFlow Variables for inputs and labels
        self.x_tf = tf.Variable(self.x)
        self.t_tf = tf.Variable(self.t)
        self.u_tf = tf.Variable(self.u)
        self.x_f_tf = tf.Variable(self.x_f)
        self.t_f_tf = tf.Variable(self.t_f)

        self.u_pred = self.net_u(self.x_tf, self.t_tf)
        self.f_pred, self.Phi_pred, self.u_t_pred = self.net_f(self.x_f_tf, self.t_f_tf, self.x_f.shape[0])

        # Losses
        self.loss_u = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))
        self.loss_f_coeff_tf = tf.Variable(1.0, dtype=tf.float32)
        self.loss_f = self.loss_f_coeff_tf * tf.reduce_mean(tf.square(self.f_pred))

        self.loss_lambda = 1e-7 * tf.norm(self.lambda1, ord=1)
        self.loss = tf.math.log(self.loss_u + self.loss_f + self.loss_lambda)  # log loss

        # Validation data
        self.x_val = X_val[:, 0:1]
        self.t_val = X_val[:, 1:2]
        self.u_val = u_val
        self.x_val_tf = tf.Variable(self.x_val)
        self.t_val_tf = tf.Variable(self.t_val)
        self.u_val_tf = tf.Variable(self.u_val)

        self.u_val_pred = self.net_u(self.x_val_tf, self.t_val_tf)
        self.f_val_pred, _, _ = self.net_f(self.x_val_tf, self.t_val_tf, self.x_val.shape[0])

        self.loss_u_val = tf.reduce_mean(tf.square(self.u_val_tf - self.u_val_pred))
        self.loss_f_val = tf.reduce_mean(tf.square(self.f_val_pred))
        self.loss_val = tf.math.log(self.loss_u_val + self.loss_f_val)  # log loss

        # Optimizer
        self.optimizer = tf.optimizers.Adam(learning_rate=1e-3)

    def initialize_NN(self, layers):
        # Initialize neural network weights and biases
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(num_layers - 1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1, layers[l+1]], dtype=tf.float32), dtype=tf.float32, name='b')
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        # Xavier initialization
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.random.normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32, name='W')

    def neural_net(self, X, weights, biases):
        # Feedforward neural network
        num_layers = len(weights) + 1
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        H = tf.cast(H, dtype=tf.float32)  # Ensure H is float32

        for l in range(num_layers - 2):
            W = weights[l]
            b = biases[l]
            W = tf.cast(W, dtype=tf.float32)  # Ensure W is float32
            b = tf.cast(b, dtype=tf.float32)  # Ensure b is float32
            H = tf.tanh(tf.add(tf.matmul(H, W), b))

        W = weights[-1]
        b = biases[-1]
        W = tf.cast(W, dtype=tf.float32)  # Ensure W is float32
        b = tf.cast(b, dtype=tf.float32)  # Ensure b is float32
        Y = tf.add(tf.matmul(H, W), b)
        return Y


    def net_u(self, x, t):
        # u(x,t) prediction
        u = self.neural_net(tf.concat([x, t], axis=1), self.weights, self.biases)
        return u

    def net_f(self, x, t, N_f):
        # Physics-based loss computation
        u = self.net_u(x, t)  # Compute network output

        # Use GradientTape to compute derivatives
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            u = self.net_u(x, t)  # Ensure forward pass is recorded

        # First derivative w.r.t t
        u_t = tape.gradient(u, t)

        # First derivative w.r.t x
        u_x = tape.gradient(u, x)

        # Second derivative w.r.t x
        u_xx = tape.gradient(u_x, x)

        # Third derivative w.r.t x
        u_xxx = tape.gradient(u_xx, x)

        # Check if any derivative is None
        if u_x is None or u_xx is None or u_xxx is None:
            raise ValueError("One of the derivatives (u_x, u_xx, u_xxx) is None. Ensure the forward pass is correctly defined.")

        # Construct the candidate library matrix (Phi)
        Phi = tf.concat([tf.ones((N_f, 1), dtype=tf.float32), u, u**2, u**3, u_x, u*u_x, u**2*u_x,
                        u**3*u_x, u_xx, u*u_xx, u**2*u_xx, u**3*u_xx, u_xxx, u*u_xxx, u**2*u_xxx, u**3*u_xxx], axis=1)

        # PDE residual: Using the library matrix and the gradient of u
        f = tf.matmul(Phi, self.lambda1) - u_t  # PDE residual
        
        return f, Phi, u_t




    
    def lbfgs_optimizer(self, loss_fn, var_list):
        """
        Performs L-BFGS-B optimization using SciPy for training the PINN.
        """
        def get_params():
            params = []
            for var in var_list:
                params.append(var.numpy().flatten())
            return np.concatenate(params)

        def loss_and_grads(params):
            offset = 0
            for var in var_list:
                param_size = np.prod(var.shape)
                var.assign(tf.convert_to_tensor(params[offset:offset + param_size].reshape(var.shape), dtype=tf.float32))
                offset += param_size
            with tf.GradientTape() as tape:
                tape.watch(var_list)
                loss_value = loss_fn()
            grads = tape.gradient(loss_value, var_list)
            grad_params = np.concatenate([grad.numpy().flatten() for grad in grads])
            return loss_value.numpy().astype(np.float64), grad_params

        initial_params = get_params()
        result = opt.minimize(loss_and_grads, initial_params, jac=True, method='L-BFGS-B', 
                              options={'maxiter': 1000, 'maxfun': 1000, 'maxcor': 50, 'maxls': 50, 
                                       'ftol': 1e-10, 'disp': True})
        return result

    def train_with_lbfgs(self, epochs=1000):
        """
        Train the Physics-Informed Neural Network using L-BFGS-B optimization.
        """
        def loss_fn():
            self.u_pred = self.net_u(self.x_tf, self.t_tf)
            self.f_pred, _, _ = self.net_f(self.x_f_tf, self.t_f_tf, self.x_f.shape[0])
            loss_u = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))
            loss_f = tf.reduce_mean(tf.square(self.f_pred))
            loss_lambda = 1e-7 * tf.norm(self.lambda1, ord=1)
            return tf.math.log(loss_u + loss_f + loss_lambda)
        
        self.lbfgs_result = self.lbfgs_optimizer(loss_fn, self.weights + self.biases + [self.lambda1])

    def adam_optimizer(self, loss_fn, var_list):
        """
        Adam optimizer used for training the PINN with the loss function.
        """
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        
        with tf.GradientTape() as tape:
            tape.watch(var_list)
            loss_value = loss_fn()

        grads = tape.gradient(loss_value, var_list)
        optimizer.apply_gradients(zip(grads, var_list))
        return loss_value

    def train(self, epochs=1000):
        """
        Train the model using both Adam and L-BFGS optimizers.
        """
        for epoch in range(epochs):
            loss_value = self.adam_optimizer(self.loss_fn, self.weights + self.biases + [self.lambda1])
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss_value.numpy()}")
            
            if epoch % 100 == 0:
                self.train_with_lbfgs()
        
    def loss_fn(self):
        """
        Compute the loss for the training process.
        """
        self.u_pred = self.net_u(self.x_tf, self.t_tf)
        self.f_pred, _, _ = self.net_f(self.x_f_tf, self.t_f_tf, self.x_f.shape[0])
        loss_u = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))
        loss_f = tf.reduce_mean(tf.square(self.f_pred))
        loss_lambda = 1e-7 * tf.norm(self.lambda1, ord=1)
        return tf.math.log(loss_u + loss_f + loss_lambda)
    
    # Callback function for Pretraining
    def callback_Pretrain(self, loss, loss_u, loss_f, loss_lambda, loss_val, loss_u_val, loss_f_val, lamu):
        global step_Pretrain
        step_Pretrain += 1
        if step_Pretrain % 10 == 0:
            print('Step: %d, log Loss: %e, loss_u: %e, loss_f: %e, loss_lambda: %e' % (step_Pretrain, loss, loss_u, loss_f,
                                                                                   loss_lambda))
            
            # Update loss history
            global loss_history_Pretrain
            global loss_u_history_Pretrain
            global loss_f_history_Pretrain
            global loss_lambda_history_Pretrain
            
            global loss_history_val_Pretrain
            global loss_u_history_val_Pretrain
            global loss_f_history_val_Pretrain
            
            global lambda_history_Pretrain
            
            loss_history_Pretrain = np.append(loss_history_Pretrain, loss)
            loss_u_history_Pretrain = np.append(loss_u_history_Pretrain, loss_u)
            loss_f_history_Pretrain = np.append(loss_f_history_Pretrain, loss_f)
            loss_lambda_history_Pretrain = np.append(loss_lambda_history_Pretrain, loss_lambda)
            
            loss_history_val_Pretrain = np.append(loss_history_val_Pretrain, loss_val)
            loss_u_history_val_Pretrain = np.append(loss_u_history_val_Pretrain, loss_u_val)
            loss_f_history_val_Pretrain = np.append(loss_f_history_val_Pretrain, loss_f_val)
            
            # Update lambda history
            lambda_history_Pretrain = np.append(lambda_history_Pretrain, lamu, axis=1)

    def train(self, nIter):  # nIter is the number of ADO loop
        self.tf_dict = {
            self.x_tf: self.x, 
            self.t_tf: self.t, 
            self.u_tf: self.u, 
            self.x_f_tf: self.x_f, 
            self.t_f_tf: self.t_f,
            self.x_val_tf: self.x_val, 
            self.t_val_tf: self.t_val, 
            self.u_val_tf: self.u_val,
            self.loss_f_coeff_tf: 1
        }

        # Pretraining, as a form of a good initialization
        print('L-BFGS-B pretraining begins')
        self.optimizer_Pretrain.minimize(
            self.loss,
            var_list=self.model.trainable_variables,
            feed_dict=self.tf_dict,
            loss_callback=self.callback_Pretrain
        )

        self.tf_dict[self.loss_f_coeff_tf] = 2
        for self.it in range(nIter):
            
            # Loop of STRidge optimization
            print('STRidge begins')
            self.callTrainSTRidge()

            # Loop of Adam optimization
            print('Adam begins')
            start_time = time.time()
            for it_Adam in range(1000):

                # Run the Adam optimization step
                self.optimizer_Adam.minimize(self.loss, var_list=self.model.trainable_variables)

                # Print every 10 steps
                if it_Adam % 10 == 0:
                    elapsed = time.time() - start_time
                    loss, loss_u, loss_f, loss_lambda, lambda1_value, loss_val, loss_u_val, loss_f_val = self.sess.run([
                        self.loss, self.loss_u, self.loss_f, self.loss_lambda, self.lambda1, self.loss_val, 
                        self.loss_u_val, self.loss_f_val
                    ], self.tf_dict)
                    print('It: %d, Log Loss: %.3e, loss_u: %e, loss_f: %e, loss_lambda: %e, Time: %.2f' 
                        % (it_Adam, loss, loss_u, loss_f, loss_lambda, elapsed))

                    lamu = self.sess.run(self.lambda1)

                    global loss_history_Adam
                    global lambda_history_Adam
                    global loss_u_history_Adam
                    global loss_f_history_Adam
                    global loss_lambda_history_Adam
                    
                    global loss_history_Adam_val
                    global loss_u_history_Adam_val
                    global loss_f_history_Adam_val

                    # Record the losses and parameters
                    loss_history_Adam = np.append(loss_history_Adam, loss)
                    lambda_history_Adam = np.append(lambda_history_Adam, lambda1_value, axis=1)
                    loss_u_history_Adam = np.append(loss_u_history_Adam, loss_u)
                    loss_f_history_Adam = np.append(loss_f_history_Adam, loss_f)
                    loss_lambda_history_Adam = np.append(loss_lambda_history_Adam, loss_lambda)

                    loss_history_Adam_val = np.append(loss_history_Adam_val, loss_val)
                    loss_u_history_Adam_val = np.append(loss_u_history_Adam_val, loss_u_val)
                    loss_f_history_Adam_val = np.append(loss_f_history_Adam_val, loss_f_val)

                    lambda_history_Adam = np.append(lambda_history_Adam, lamu, axis=1)

                    start_time = time.time()

            # Loop of L-BFGS-B optimization
            # print('L-BFGS-B begins')
            # self.optimizer.minimize(self.sess,
            #                         feed_dict = self.tf_dict,
            #                         fetches = [self.loss, self.loss_u, self.loss_f, self.loss_lambda,
            #                                    self.loss_val, self.loss_u_val, self.loss_f_val],
            #                         loss_callback = self.callback)
            
        # One more time of STRidge optimization
        print('STRidge begins')
        self.callTrainSTRidge()

    
    def predict(self, X_star):
        tf_dict = {self.x_tf: X_star[:, 0:1], self.t_tf: X_star[:, 1:2]}
        u_star = self.sess.run(self.u_pred, tf_dict)
        return u_star

    def callTrainSTRidge(self):
        lam = 1e-5
        d_tol = 1
        maxit = 100
        STR_iters = 10

        l0_penalty = None

        normalize = 2
        split = 0.8
        print_best_tol = False
        Phi_pred, u_t_pred = self.sess.run([self.Phi_pred, self.u_t_pred], self.tf_dict)

        lambda2 = self.TrainSTRidge(Phi_pred, u_t_pred, lam, d_tol, maxit, STR_iters, l0_penalty, normalize, split,
                                     print_best_tol)

        self.lambda1.assign(tf.convert_to_tensor(lambda2, dtype=tf.float32))

    def TrainSTRidge(self, R0, Ut, lam, d_tol, maxit, STR_iters=10, l0_penalty=None, normalize=2, split=0.8,
                     print_best_tol=False):
        # Normalization logic as in the original code
        n, d = R0.shape
        R = np.zeros((n, d), dtype=np.float32)
        if normalize != 0:
            Mreg = np.zeros((d, 1))
            for i in range(0, d):
                Mreg[i] = 1.0 / (np.linalg.norm(R0[:, i], normalize))
                R[:, i] = Mreg[i] * R0[:, i]
            normalize_inner = 0
        else:
            R = R0
            Mreg = np.ones((d, 1)) * d
            normalize_inner = 2

        # Split data
        np.random.seed(0)
        n, _ = R.shape
        train = np.random.choice(n, int(n * split), replace=False)
        test = [i for i in np.arange(n) if i not in train]
        TrainR = R[train, :]
        TestR = R[test, :]
        TrainY = Ut[train, :]
        TestY = Ut[test, :]

        # Setup tolerance
        d_tol = float(d_tol)
        if self.it == 0:
            self.tol = d_tol

        # Or inherit Lambda
        w_best = self.sess.run(self.lambda1) / Mreg
        err_f = np.mean((TestY - TestR.dot(w_best)) ** 2)

        if l0_penalty is None and self.it == 0:
            self.l0_penalty_0 = err_f
            l0_penalty = self.l0_penalty_0
        elif l0_penalty is None:
            l0_penalty = self.l0_penalty_0

        err_lambda = l0_penalty * np.count_nonzero(w_best)
        err_best = err_f + err_lambda
        tol_best = 0

        # Update loss history
        loss_history_STRidge = np.append(loss_history_STRidge, err_best)
        loss_f_history_STRidge = np.append(loss_f_history_STRidge, err_f)
        loss_lambda_history_STRidge = np.append(loss_lambda_history_STRidge, err_lambda)
        tol_history_STRidge = np.append(tol_history_STRidge, tol_best)

        # Increase tolerance until test performance decreases
        for iter in range(maxit):
            # Get coefficients and error
            w = self.STRidge(TrainR, TrainY, lam, STR_iters, self.tol, Mreg, normalize=normalize_inner)

            err_f = np.mean((TestY - TestR.dot(w)) ** 2)
            err_lambda = l0_penalty * np.count_nonzero(w)
            err = err_f + err_lambda

            # If accuracy improves, update
            if err <= err_best:
                err_best = err
                w_best = w
                tol_best = self.tol
                self.tol = self.tol + d_tol

                loss_history_STRidge = np.append(loss_history_STRidge, err_best)
                loss_f_history_STRidge = np.append(loss_f_history_STRidge, err_f)
                loss_lambda_history_STRidge = np.append(loss_lambda_history_STRidge, err_lambda)
                tol_history_STRidge = np.append(tol_history_STRidge, tol_best)
            else:
                self.tol = max([0, self.tol - 2 * d_tol])
                d_tol = d_tol / 1.618
                self.tol = self.tol + d_tol

        if print_best_tol: print("Optimal tolerance:", tol_best)

        optimaltol_history = np.append(optimaltol_history, tol_best)

        return np.real(np.multiply(Mreg, w_best))
    

    def STRidge(self, X0, y, lam, maxit, tol, Mreg, normalize=2, print_results=False):
        n, d = X0.shape
        X = np.zeros((n, d), dtype=np.complex64)
        
        # First normalize data
        if normalize != 0:
            Mreg = np.zeros((d, 1))
            for i in range(d):
                Mreg[i] = 1.0 / (np.linalg.norm(X0[:, i], normalize))
                X[:, i] = Mreg[i] * X0[:, i]                
        else: 
            X = X0
        
        # Inherit lambda
        w = self.lambda1.numpy() / Mreg  # TensorFlow 2.x, use .numpy() to extract values from tensors

        biginds = np.where(abs(w) > tol)[0]
        num_relevant = d            
        
        # Global variable initialization
        global ridge_append_counter_STRidge
        ridge_append_counter = 0
        
        global lambda_history_STRidge
        lambda_history_STRidge = np.append(lambda_history_STRidge, np.multiply(Mreg, w), axis=1)
        ridge_append_counter += 1
        
        # Threshold and continue
        for j in range(maxit):
            # Figure out which items to cut out
            smallinds = np.where(abs(w) < tol)[0]
            new_biginds = [i for i in range(d) if i not in smallinds]
                    
            # If nothing changes then stop
            if num_relevant == len(new_biginds): break
            else: num_relevant = len(new_biginds)
                    
            if len(new_biginds) == 0:
                if j == 0: 
                    if normalize != 0:
                        lambda_history_STRidge = np.append(lambda_history_STRidge, w * Mreg, axis=1)
                        ridge_append_counter += 1
                        ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
                        return np.multiply(Mreg, w)
                    else:
                        lambda_history_STRidge = np.append(lambda_history_STRidge, w * Mreg, axis=1)
                        ridge_append_counter += 1
                        ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
                        return w
                else: break
            biginds = new_biginds
            
            # Otherwise get a new guess
            w[smallinds] = 0
            
            if lam != 0: 
                # Solve using least squares with regularization
                X_biginds = X[:, biginds]
                w[biginds] = np.linalg.lstsq(X_biginds.T.dot(X_biginds) + lam * np.eye(len(biginds)), X_biginds.T.dot(y))[0]
                lambda_history_STRidge = np.append(lambda_history_STRidge, np.multiply(Mreg, w), axis=1)
                ridge_append_counter += 1
            else: 
                # Solve without regularization
                w[biginds] = np.linalg.lstsq(X[:, biginds], y)[0]
                lambda_history_STRidge = np.append(lambda_history_STRidge, np.multiply(Mreg, w), axis=1)
                ridge_append_counter += 1

        # Now that we have the sparsity pattern, use standard least squares to get w
        if biginds != []:
            w[biginds] = np.linalg.lstsq(X[:, biginds], y)[0]
            
        if normalize != 0:
            lambda_history_STRidge = np.append(lambda_history_STRidge, w * Mreg, axis=1)
            ridge_append_counter += 1
            ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
            return np.multiply(Mreg, w)
        else:
            lambda_history_STRidge = np.append(lambda_history_STRidge, w * Mreg, axis=1)
            ridge_append_counter += 1
            ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
            return w


    



In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
import os
from scipy.interpolate import griddata
from scipy.spatial import distance
import time
from mpl_toolkits.mplot3d import Axes3D
from pyDOE import lhs

# Define layers for the neural network
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

# Load the data
data = scipy.io.loadmat("C:/Users/chidi/Downloads/burgers.mat")

t = np.real(data['t'].flatten()[:, None])
x = np.real(data['x'].flatten()[:, None])
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

# Measurement data
N_u_s = 10
idx_s = np.random.choice(x.shape[0], N_u_s, replace=False)
X0 = X[:, idx_s]
T0 = T[:, idx_s]
Exact0 = Exact[:, idx_s]

N_u_t = int(t.shape[0] * 1)
idx_t = np.random.choice(t.shape[0], N_u_t, replace=False)
X0 = X0[idx_t, :]
T0 = T0[idx_t, :]
Exact0 = Exact0[idx_t, :]

X_u_meas = np.hstack((X0.flatten()[:, None], T0.flatten()[:, None]))
u_meas = Exact0.flatten()[:, None]

# Training measurements, randomly sampled spatio-temporally
Split_TrainVal = 0.8
N_u_train = int(X_u_meas.shape[0] * Split_TrainVal)
idx_train = np.random.choice(X_u_meas.shape[0], N_u_train, replace=False)
X_u_train = X_u_meas[idx_train, :]
u_train = u_meas[idx_train, :]

# Validation Measurements
idx_val = np.setdiff1d(np.arange(X_u_meas.shape[0]), idx_train, assume_unique=True)
X_u_val = X_u_meas[idx_val, :]
u_val = u_meas[idx_val, :]

# Collocation points
N_f = 50000
X_f_train = lb + (ub - lb) * lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))

"""
# Option: Add noise
noise = 0.1
u_train = u_train + noise * np.std(u_train) * np.random.randn(u_train.shape[0], u_train.shape[1])
u_val = u_val + noise * np.std(u_val) * np.random.randn(u_val.shape[0], u_val.shape[1])
"""
# Model initialization
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, X_u_val, u_val, layers, lb, ub)
model.train(6)


TypeError: Argument `target` should be a list or nested structure of Tensors, Variables or CompositeTensors to be differentiated, but received None.

In [ ]:
# %% [markdown]
# ## Physics-Informed Neural Networks for Burgers' Equation
# This notebook demonstrates the discovery of the Burgers' equation using a physics-informed neural network (PINN).

# %%
# Import necessary libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from scipy.spatial import distance
from matplotlib import cm
from pyDOE import lhs
import os
import time

# %%
# Set TensorFlow to execute eagerly
tf.compat.v1.disable_eager_execution()

# %%
# Define the PhysicsInformedNN class
class PhysicsInformedNN:
    def __init__(self, X, u, X_f, X_val, u_val, layers, lb, ub):
        """
        Initializes the Physics-Informed Neural Network (PINN).
        """
        self.lb = lb
        self.ub = ub
        self.layers = layers

        # Initialize NN weights and biases
        self.weights, self.biases = self.initialize_NN(layers)

        # Input placeholders
        self.x_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, X.shape[1]])
        self.u_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, u.shape[1]])
        self.x_f_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, X_f.shape[1]])
        self.u_val_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, u_val.shape[1]])

        # Prediction for u
        self.u_pred = self.neural_net(self.x_tf)

        # Physics loss components
        self.f_pred = self.physics_loss(self.x_f_tf)

        # Losses
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + tf.reduce_mean(tf.square(self.f_pred))

        # Optimizers
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
        self.train_op = self.optimizer.minimize(self.loss)

        # TensorFlow session
        self.sess = tf.compat.v1.Session()
        self.sess.run(tf.compat.v1.global_variables_initializer())

    def initialize_NN(self, layers):
        """
        Xavier initialization of weights and biases for the neural network.
        """
        weights = []
        biases = []
        for i in range(len(layers) - 1):
            W = tf.Variable(tf.random.truncated_normal([layers[i], layers[i + 1]], stddev=0.1), dtype=tf.float32)
            b = tf.Variable(tf.zeros([1, layers[i + 1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def neural_net(self, x):
        """
        Forward pass through the neural network.
        """
        H = 2.0 * (x - self.lb) / (self.ub - self.lb) - 1.0
        for i in range(len(self.weights) - 1):
            H = tf.nn.tanh(tf.add(tf.matmul(H, self.weights[i]), self.biases[i]))
        return tf.add(tf.matmul(H, self.weights[-1]), self.biases[-1])

    def physics_loss(self, x):
        """
        Computes the physics-based loss.
        """
        u = self.neural_net(x)
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        return -0.01 * u_xx + u * u_x

    def train(self, nIter):
        """
        Training loop for the PINN.
        """
        for it in range(nIter):
            u = self.neural_net(x)
            self.sess.run(self.train_op, feed_dict={self.x_tf: X, self.u_tf: u, self.x_f_tf: X_f})
            if it % 100 == 0:
                loss_value = self.sess.run(self.loss, feed_dict={self.x_tf: X, self.u_tf: u, self.x_f_tf: X_f})
                print(f"Iteration {it}: Loss = {loss_value}")

# %%
# Load data
data = scipy.io.loadmat("data/burgers.mat")
t = data["t"].flatten()[:, None]
x = data["x"].flatten()[:, None]
Exact = np.real(data["usol"]).T

# Domain bounds
X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]
lb = X_star.min(0)
ub = X_star.max(0)

# %%
# Generate training data
N_u = 2000
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx, :]

# Generate collocation points
N_f = 10000
X_f_train = lb + (ub - lb) * lhs(2, N_f)

# %%
# Define and train the model
layers = [2, 50, 50, 50, 1]
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, X_u_train, u_train, layers, lb, ub)
model.train(1000)

# %%
# Predict and visualize results
u_pred = model.sess.run(model.u_pred, feed_dict={model.x_tf: X_star})
u_pred = griddata(X_star, u_pred.flatten(), (X, T), method="cubic")

plt.figure(figsize=(9, 6))
plt.pcolor(T, X, u_pred, cmap=cm.coolwarm)
plt.colorbar()
plt.xlabel("t")
plt.ylabel("x")
plt.title("Predicted Solution")
plt.show()


NameError: name 'u' is not defined

In [ ]:
# %% [markdown]
# ## Physics-Informed Neural Network (PINN) for PDE Discovery
# This notebook implements a PINN for discovering governing equations such as the Burgers' equation.

# %%
# Import necessary libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from pyDOE import lhs
import time
import os

# %%
# Disable eager execution for compatibility with TensorFlow v1 code style
tf.compat.v1.disable_eager_execution()

# %%
# Define the PhysicsInformedNN class
# Define the PhysicsInformedNN class
class PhysicsInformedNN:
    def __init__(self, X, u, X_f, X_val, u_val, layers, lb, ub):
        """
        Initializes the Physics-Informed Neural Network (PINN).
        """
        self.lb = tf.constant(lb.reshape(1, -1), dtype=tf.float32)  # Reshape to [1, n_features]
        self.ub = tf.constant(ub.reshape(1, -1), dtype=tf.float32)  # Reshape to [1, n_features]
        self.layers = layers

        # Initialize NN weights and biases
        self.weights, self.biases = self.initialize_NN(layers)

        # TensorFlow placeholders for inputs and outputs
        self.x_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, X.shape[1]])
        self.t_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, 1])
        self.u_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, 1])
        self.x_f_tf = tf.compat.v1.placeholder(tf.float32, shape=[None, X_f.shape[1]])

        # Neural network predictions
        self.u_pred = self.net_u(self.x_tf, self.t_tf)
        self.f_pred, self.Phi_pred, self.u_t_pred = self.net_f(self.x_f_tf)

        # Loss function
        self.loss_u = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))
        self.loss_f = tf.reduce_mean(tf.square(self.f_pred))
        self.loss = self.loss_u + self.loss_f

        # Optimizer
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)
        self.train_op = self.optimizer.minimize(self.loss)

        # TensorFlow session
        self.sess = tf.compat.v1.Session()
        self.sess.run(tf.compat.v1.global_variables_initializer())

    def initialize_NN(self, layers):
        """
        Xavier initialization of weights and biases for the neural network.
        """
        weights = []
        biases = []
        for i in range(len(layers) - 1):
            W = tf.Variable(tf.random.truncated_normal([layers[i], layers[i + 1]], stddev=0.1), dtype=tf.float32)
            b = tf.Variable(tf.zeros([1, layers[i + 1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def neural_net(self, X, weights, biases):
        """
        Forward pass through the neural network.
        """
        # Debugging input shapes
        tf.print("Shape of X:", tf.shape(X))
        tf.print("Shape of lb:", tf.shape(self.lb))
        tf.print("Shape of ub:", tf.shape(self.ub))

        # Reshape lb and ub to be broadcastable
        self.lb = tf.reshape(self.lb, [1, -1])
        self.ub = tf.reshape(self.ub, [1, -1])

        # Normalize input
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0

        # Forward pass
        for i in range(len(weights) - 1):
            W = weights[i]
            b = biases[i]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        return tf.add(tf.matmul(H, W), b)


    def net_u(self, x, t):
        """
        Forward pass for the solution u(x,t).
        """
        return self.neural_net(tf.concat([x, t], axis=1), self.weights, self.biases)

    def net_f(self, X_f):
        """
        Physics-based loss for the PDE residual and candidate library construction.
        """
        x, t = X_f[:, 0:1], X_f[:, 1:2]
        u = self.net_u(x, t)

        # Derivatives
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]

        # Candidate library Phi
        Phi = tf.concat([tf.ones_like(u), u, u**2, u_x, u**3, u_xx], axis=1)

        # Residual
        f = u_t + u * u_x - (0.01 / np.pi) * u_xx
        return f, Phi, u_t

    def train(self, nIter, X_u_train, u_train, X_f_train):
        """
        Training loop for the PINN.
        """
        tf_dict = {self.x_tf: X_u_train[:, 0:1], self.t_tf: X_u_train[:, 1:2], self.u_tf: u_train,
                   self.x_f_tf: X_f_train}

        for it in range(nIter):
            self.sess.run(self.train_op, tf_dict)

            if it % 10 == 0:
                loss_value = self.sess.run(self.loss, tf_dict)
                print(f"Iteration {it}, Loss: {loss_value}")


In [ ]:
# %%
# Load and preprocess data
data = scipy.io.loadmat("data/burgers.mat")
t = data["t"].flatten()[:, None]
x = data["x"].flatten()[:, None]
Exact = np.real(data["usol"]).T

X, T = np.meshgrid(x, t)
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

# %%
# Training data
N_u = 2000
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx, :]

# Collocation points
N_f = 10000
X_f_train = lb + (ub - lb) * lhs(2, N_f)

# %%
# Define the PINN model
layers = [2, 50, 50, 50, 50, 1]
model = PhysicsInformedNN(X_u_train, u_train, X_f_train, X_u_train, u_train, layers, lb, ub)

# Train the model
model.train(1000, X_u_train, u_train, X_f_train)

# Discover PDE
lambda_discovered = model.callTrainSTRidge()
print("Discovered PDE coefficients:", lambda_discovered)


ValueError: Dimensions must be equal, but are 3 and 2 for '{{node sub_8}} = Sub[T=DT_FLOAT](concat_3, Reshape)' with input shapes: [?,3], [1,2].

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from scipy.spatial import distance
from matplotlib import cm
import time
from mpl_toolkits.mplot3d import Axes3D
from pyDOE import lhs
#    import sobol_seq
import os

# Set device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss histories to record convergence
# Adam loss history
loss_history_Adam = np.array([0])
loss_u_history_Adam = np.array([0])
loss_f_history_Adam = np.array([0])
loss_lambda_history_Adam = np.array([0])
lambda_history_Adam = np.zeros((16, 1))
loss_history_Adam_val = np.array([0])
loss_u_history_Adam_val = np.array([0])
loss_f_history_Adam_val = np.array([0])

# STRidge loss history
loss_history_STRidge = np.array([0])
loss_f_history_STRidge = np.array([0])
loss_lambda_history_STRidge = np.array([0])
optimaltol_history = np.array([0])
tol_history_STRidge = np.array([0])
lambda_normalized_history_STRidge = np.zeros((16, 1))

lambda_history_STRidge = np.zeros((16, 1))
ridge_append_counter_STRidge = np.array([0])

# Loss histories for pretraining
loss_history_Pretrain = np.array([0])
loss_u_history_Pretrain = np.array([0])
loss_f_history_Pretrain = np.array([0])
loss_lambda_history_Pretrain = np.array([0])
loss_history_val_Pretrain = np.array([0])
loss_u_history_val_Pretrain = np.array([0])
loss_f_history_val_Pretrain = np.array([0])
step_Pretrain = 0

lambda_history_Pretrain = np.zeros((16, 1))

# Set the random seeds for reproducibility
np.random.seed(1234)
torch.manual_seed(1234)

# Add other PyTorch-specific functionality as needed

# Example of creating a tensor and moving it to GPU
# x = torch.tensor(np.random.randn(10, 1), dtype=torch.float32, requires_grad=True).to(device)



In [ ]:
# Set device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============================================================================
# Define the Physics-Informed Neural Network class (PINN)
# =============================================================================
class PhysicsInformedNN:
    def __init__(self, X, u, X_f, X_val, u_val, layers, lb, ub):
        self.lb = lb
        self.ub = ub
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Training data
        self.x = torch.tensor(X[:, 0:1], dtype=torch.float32, requires_grad=True).to(device)
        self.t = torch.tensor(X[:, 1:2], dtype=torch.float32, requires_grad=True).to(device)
        self.u = torch.tensor(u, dtype=torch.float32).to(device)
        
        # Collocation data
        self.x_f = torch.tensor(X_f[:, 0:1], dtype=torch.float32, requires_grad=True).to(device)
        self.t_f = torch.tensor(X_f[:, 1:2], dtype=torch.float32, requires_grad=True).to(device)
        
        # Validation data
        self.x_val = torch.tensor(X_val[:, 0:1], dtype=torch.float32).to(device)
        self.t_val = torch.tensor(X_val[:, 1:2], dtype=torch.float32).to(device)
        self.u_val = torch.tensor(u_val, dtype=torch.float32).to(device)
        
        # Initializing lambda
        self.lambda1 = torch.zeros((16, 1), dtype=torch.float32, requires_grad=True).to(device)
        
        # Loss function placeholders
        self.loss_u = torch.nn.MSELoss()
        
        # Placeholder for physics-based loss
        self.loss_f_coeff = torch.tensor(1.0, dtype=torch.float32, requires_grad=False).to(device)
        
        self.optimizer_Adam = torch.optim.Adam([{'params': self.weights}, {'params': self.biases}, {'params': self.lambda1}], lr=1e-3)
        
        # Model forward pass and loss computation
        self.u_pred = self.net_u(self.x, self.t)
        self.f_pred, self.Phi_pred, self.u_t_pred = self.net_f(self.x_f, self.t_f)
        
        self.loss_u_val = self.loss_u(self.u_pred, self.u)
        self.loss_f = self.loss_f_coeff * torch.mean(torch.square(self.f_pred))
        self.loss_lambda = 1e-7 * torch.norm(self.lambda1, p=1)
        
        # Total loss
        self.loss = torch.log(self.loss_u_val + self.loss_f + self.loss_lambda)
        
        # Validation loss
        self.u_val_pred = self.net_u(self.x_val, self.t_val)
        self.loss_u_val = self.loss_u(self.u_val_pred, self.u_val)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0, num_layers - 1):
            W = self.xavier_init(layers[l], layers[l + 1])
            b = torch.zeros(1, layers[l + 1], dtype=torch.float32).to(device)
            weights.append(W)
            biases.append(b)
        return weights, biases
        
    def xavier_init(self, in_dim, out_dim):
        xavier_stddev = np.sqrt(2.0 / (in_dim + out_dim))
        return torch.randn((in_dim, out_dim), dtype=torch.float32).to(device) * xavier_stddev
    
    def neural_net(self, X, weights, biases):
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for l in range(len(weights) - 1):
            W = weights[l]
            b = biases[l]
            H = torch.tanh(torch.add(torch.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = torch.add(torch.matmul(H, W), b)
        return Y
        
    def net_u(self, x, t):
        return self.neural_net(torch.cat([x, t], dim=1), self.weights, self.biases)
    
    def net_f(self, x, t):
        u = self.net_u(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        u_xxx = torch.autograd.grad(u_xx, x, grad_outputs=torch.ones_like(u_xx), create_graph=True)[0]
        
        Phi = torch.cat([torch.ones_like(u), u, u**2, u**3, u_x, u*u_x, u**2*u_x, u**3*u_x, u_xx, u*u_xx, u**2*u_xx, u**3*u_xx, u_xxx, u*u_xxx, u**2*u_xxx, u**3*u_xxx], dim=1)
        
        f = u_t - self.lambda1[0] * u - self.lambda1[1] * u_xx
        
        return f, Phi, u_t

    def train(self, nIter, X_u_train, u_train, X_f_train):
        for epoch in range(nIter):
            self.optimizer_Adam.zero_grad()
            loss = self.loss
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item()}")

    def callback_Pretrain(self, loss, loss_u, loss_f, loss_lambda, loss_val, loss_u_val, loss_f_val, lamu):
        step_Pretrain = 0
        step_Pretrain += 1
        if step_Pretrain % 10 == 0:
            print(f'Step: {step_Pretrain}, log Loss: {loss:.3e}, loss_u: {loss_u:.3e}, loss_f: {loss_f:.3e}, loss_lambda: {loss_lambda:.3e}')
            
            # Save losses and lambda values in lists (history)
            loss_history_Pretrain
            loss_u_history_Pretrain
            loss_f_history_Pretrain
            loss_lambda_history_Pretrain
            loss_history_val_Pretrain
            loss_u_history_val_Pretrain
            loss_f_history_val_Pretrain
            lambda_history_Pretrain
            
            loss_history_Pretrain.append(loss)
            loss_u_history_Pretrain.append(loss_u)
            loss_f_history_Pretrain.append(loss_f)
            loss_lambda_history_Pretrain.append(loss_lambda)
            loss_history_val_Pretrain.append(loss_val)
            loss_u_history_val_Pretrain.append(loss_u_val)
            loss_f_history_val_Pretrain.append(loss_f_val)
            lambda_history_Pretrain.append(lamu)

    def train(self, nIter):  # nIter is the number of iterations
        self.loss_f_coeff = torch.tensor(1.0, dtype=torch.float32, requires_grad=False)
        
        # Pretraining phase using L-BFGS-B optimizer equivalent in PyTorch (via scipy)
        print('L-BFGS-B pretraining begins')
        self.optimizer_Adam.zero_grad()  # Zero gradients
        
        for epoch in range(nIter):
            # Perform STRidge optimization (custom method)
            print('STRidge begins')
            self.callTrainSTRidge()
    
            # Adam optimization loop
            print('Adam begins')
            start_time = time.time()
            for it_Adam in range(1000):
                self.optimizer_Adam.zero_grad()  # Zero gradients before backward pass
                loss = self.loss  # Your loss function here
                loss.backward()  # Backpropagate gradients
                self.optimizer_Adam.step()  # Update weights

                # Print progress every 10 iterations
                if it_Adam % 10 == 0:
                    elapsed = time.time() - start_time
                    print(f'It: {it_Adam}, Log Loss: {loss:.3e}, Time: {elapsed:.2f}')
                    
                    lamu = self.lambda1.detach().cpu().numpy()  # Get current lambda values
                    
                    # Save history (loss, lambda, etc.)
                    loss_history_Adam
                    lambda_history_Adam
                    loss_u_history_Adam
                    loss_f_history_Adam
                    loss_lambda_history_Adam
                    loss_history_Adam_val
                    loss_u_history_Adam_val
                    loss_f_history_Adam_val
                    
                    loss_history_Adam.append(loss.item())
                    lambda_history_Adam.append(lamu)
                    # Add other losses to history
                    loss_history_Adam_val.append(loss_val.item())
                    loss_u_history_Adam_val.append(loss_u_val.item())
                    loss_f_history_Adam_val.append(loss_f_val.item())
            
            start_time = time.time()

        # One more round of STRidge optimization after Adam
        print('STRidge begins')
        self.callTrainSTRidge()


    def predict(self, X_star):
        x_star = torch.tensor(X_star[:, 0:1], dtype=torch.float32, requires_grad=True)
        t_star = torch.tensor(X_star[:, 1:2], dtype=torch.float32, requires_grad=True)
        u_star = self.net_u(x_star, t_star)
        return u_star.detach().numpy()

    def callTrainSTRidge(self):
        lam = 1e-5
        d_tol = 1
        maxit = 100
        STR_iters = 10
        l0_penalty = None
        normalize = 2
        split = 0.8
        print_best_tol = False

        with torch.no_grad():
            Phi_pred, u_t_pred = self.net_f(self.x_f, self.t_f)

        lambda2 = self.TrainSTRidge(Phi_pred, u_t_pred, lam, d_tol, maxit, STR_iters, l0_penalty, normalize, split, print_best_tol)
        self.lambda1.data = torch.tensor(lambda2, dtype=torch.float32)

    def TrainSTRidge(self, R0, Ut, lam, d_tol, maxit, STR_iters=10, l0_penalty=None, normalize=2, split=0.8, print_best_tol=False):
        n, d = R0.shape
        R = torch.zeros((n, d), dtype=torch.float32)

        if normalize != 0:
            Mreg = torch.zeros((d, 1), dtype=torch.float32)
            for i in range(d):
                Mreg[i] = 1.0 / torch.norm(R0[:, i], p=normalize)
                R[:, i] = Mreg[i] * R0[:, i]
            normalize_inner = 0
        else:
            R = R0.clone()
            Mreg = torch.ones((d, 1), dtype=torch.float32) * d
            normalize_inner = 2

        lambda_normalized_history_STRidge
        lambda_normalized_history_STRidge = np.append(lambda_normalized_history_STRidge, Mreg.numpy(), axis=1)

        # Split data into training and test sets
        np.random.seed(0)
        indices = np.random.permutation(n)
        train_size = int(n * split)
        train_idx = indices[:train_size]
        test_idx = indices[train_size:]

        TrainR = R[train_idx, :]
        TestR = R[test_idx, :]
        TrainY = Ut[train_idx, :]
        TestY = Ut[test_idx, :]

        # Set up the initial tolerance and l0 penalty
        d_tol = float(d_tol)
        if not hasattr(self, "tol"):
            self.tol = d_tol

        w_best = (self.lambda1 / Mreg).detach()
        err_f = torch.mean((TestY - TestR @ w_best) ** 2)

        if l0_penalty is None and not hasattr(self, "l0_penalty_0"):
            self.l0_penalty_0 = err_f
            l0_penalty = self.l0_penalty_0
        elif l0_penalty is None:
            l0_penalty = self.l0_penalty_0

        err_lambda = l0_penalty * (w_best != 0).sum().item()
        err_best = err_f + err_lambda
        tol_best = 0

        loss_history_STRidge
        loss_f_history_STRidge
        loss_lambda_history_STRidge
        tol_history_STRidge

        loss_history_STRidge = np.append(loss_history_STRidge, err_best.numpy())
        loss_f_history_STRidge = np.append(loss_f_history_STRidge, err_f.numpy())
        loss_lambda_history_STRidge = np.append(loss_lambda_history_STRidge, err_lambda)
        tol_history_STRidge = np.append(tol_history_STRidge, tol_best)

        # Increase tolerance until test performance decreases
        for _ in range(maxit):
            w = self.STRidge(TrainR, TrainY, lam, STR_iters, self.tol, Mreg, normalize_inner)
            err_f = torch.mean((TestY - TestR @ w) ** 2)
            err_lambda = l0_penalty * (w != 0).sum().item()
            err = err_f + err_lambda

            if err <= err_best:
                err_best = err
                w_best = w
                tol_best = self.tol
                self.tol += d_tol

                loss_history_STRidge = np.append(loss_history_STRidge, err_best.numpy())
                loss_f_history_STRidge = np.append(loss_f_history_STRidge, err_f.numpy())
                loss_lambda_history_STRidge = np.append(loss_lambda_history_STRidge, err_lambda)
                tol_history_STRidge = np.append(tol_history_STRidge, tol_best)
            else:
                self.tol = max(0, self.tol - 2 * d_tol)
                d_tol /= 1.618
                self.tol += d_tol

        if print_best_tol:
            print("Optimal tolerance:", tol_best)

        optimaltol_history
        optimaltol_history = np.append(optimaltol_history, tol_best)

        return (w_best * Mreg).numpy()

    
    def STRidge(self, X0, y, lam, maxit, tol, Mreg, normalize=2, print_results=False):
        """
        Sequential Threshold Ridge Regression (STRidge)
        Converted to PyTorch from TensorFlow/Numpy.
        """
        n, d = X0.shape
        X = torch.zeros((n, d), dtype=torch.complex64)

        # Normalize data
        if normalize != 0:
            Mreg = torch.zeros((d, 1), dtype=torch.float32)
            for i in range(d):
                Mreg[i] = 1.0 / torch.norm(X0[:, i], p=normalize)
                X[:, i] = Mreg[i] * X0[:, i]
        else:
            X = X0.clone()

        # Inherit lambda
        w = self.lambda1.detach() / Mreg

        # Select indices for big weights
        biginds = torch.where(torch.abs(w) > tol)[0]
        num_relevant = d

        # variables for logging
        ridge_append_counter_STRidge
        ridge_append_counter = 0

        lambda_history_STRidge
        lambda_history_STRidge = np.append(lambda_history_STRidge, (Mreg * w).numpy(), axis=1)
        ridge_append_counter += 1

        # Threshold and iterate
        for _ in range(maxit):
            # Determine small indices and new big indices
            smallinds = torch.where(torch.abs(w) < tol)[0]
            new_biginds = [i for i in range(d) if i not in smallinds]

            # Stop if no changes
            if num_relevant == len(new_biginds):
                break
            else:
                num_relevant = len(new_biginds)

            if len(new_biginds) == 0:
                if _ == 0:
                    if normalize != 0:
                        lambda_history_STRidge = np.append(lambda_history_STRidge, (w * Mreg).numpy(), axis=1)
                        ridge_append_counter += 1
                        ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
                        return (Mreg * w).numpy()
                    else:
                        lambda_history_STRidge = np.append(lambda_history_STRidge, (w * Mreg).numpy(), axis=1)
                        ridge_append_counter += 1
                        ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
                        return w.numpy()
                else:
                    break
            biginds = new_biginds

            # Update weights
            w[smallinds] = 0

            if lam != 0:
                w[biginds] = torch.linalg.lstsq(
                    X[:, biginds].T @ X[:, biginds] + lam * torch.eye(len(biginds), dtype=torch.float32),
                    X[:, biginds].T @ y,
                )[0]
                lambda_history_STRidge = np.append(lambda_history_STRidge, (Mreg * w).numpy(), axis=1)
                ridge_append_counter += 1
            else:
                w[biginds] = torch.linalg.lstsq(X[:, biginds], y)[0]
                lambda_history_STRidge = np.append(lambda_history_STRidge, (Mreg * w).numpy(), axis=1)
                ridge_append_counter += 1

        # Final least squares with sparsity pattern
        if len(biginds) > 0:
            w[biginds] = torch.linalg.lstsq(X[:, biginds], y)[0]

        if normalize != 0:
            lambda_history_STRidge = np.append(lambda_history_STRidge, (w * Mreg).numpy(), axis=1)
            ridge_append_counter += 1
            ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
            return (Mreg * w).numpy()
        else:
            lambda_history_STRidge = np.append(lambda_history_STRidge, (w * Mreg).numpy(), axis=1)
            ridge_append_counter += 1
            ridge_append_counter_STRidge = np.append(ridge_append_counter_STRidge, ridge_append_counter)
            return w.numpy()



In [ ]:
import torch
import numpy as np
import scipy.io
from pyDOE import lhs

# Define the model architecture
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

# =============================================================================
# Load data
# =============================================================================
data = scipy.io.loadmat("C:/Users/chidi/Downloads/burgers.mat")

t = np.real(data['t'].flatten()[:, None])
x = np.real(data['x'].flatten()[:, None])
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()[:, None]

# Domain bounds
lb = torch.tensor(X_star.min(0), dtype=torch.float32)
ub = torch.tensor(X_star.max(0), dtype=torch.float32)

# Measurement data
N_u_s = 10  # Number of spatial points
idx_s = np.random.choice(x.shape[0], N_u_s, replace=False)
X0 = X[:, idx_s]
T0 = T[:, idx_s]
Exact0 = Exact[:, idx_s]

N_u_t = int(t.shape[0] * 1)  # Number of temporal points
idx_t = np.random.choice(t.shape[0], N_u_t, replace=False)
X0 = X0[idx_t, :]
T0 = T0[idx_t, :]
Exact0 = Exact0[idx_t, :]

X_u_meas = np.hstack((X0.flatten()[:, None], T0.flatten()[:, None]))
u_meas = Exact0.flatten()[:, None]

# Split data into training and validation
Split_TrainVal = 0.8
N_u_train = int(X_u_meas.shape[0] * Split_TrainVal)
idx_train = np.random.choice(X_u_meas.shape[0], N_u_train, replace=False)
X_u_train = X_u_meas[idx_train, :]
u_train = u_meas[idx_train, :]

idx_val = np.setdiff1d(np.arange(X_u_meas.shape[0]), idx_train, assume_unique=True)
X_u_val = X_u_meas[idx_val, :]
u_val = u_meas[idx_val, :]

# Collocation points
N_f = 50000
X_f_train = lb + (ub - lb) * torch.tensor(lhs(2, N_f), dtype=torch.float32)
X_f_train = torch.cat((X_f_train, torch.tensor(X_u_train, dtype=torch.float32)), dim=0)

# Option: Add noise
noise = 0.1
u_train = torch.tensor(u_train, dtype=torch.float32) + noise * torch.std(torch.tensor(u_train, dtype=torch.float32)) * torch.randn_like(torch.tensor(u_train, dtype=torch.float32))
u_val = torch.tensor(u_val, dtype=torch.float32) + noise * torch.std(torch.tensor(u_val, dtype=torch.float32)) * torch.randn_like(torch.tensor(u_val, dtype=torch.float32))

# =============================================================================
# Model training
# =============================================================================
# Initialize model (requires the PyTorch version of PhysicsInformedNN)
model = PhysicsInformedNN(
    torch.tensor(X_u_train, dtype=torch.float32),
    u_train,
    X_f_train,
    torch.tensor(X_u_val, dtype=torch.float32),
    u_val,
    layers,
    lb,
    ub
)

# Train the model
model.train(6)


C:\Users\chidi\AppData\Local\Temp\ipykernel_5856\218216156.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(X[:, 0:1], dtype=torch.float32, requires_grad=True).to(device)
C:\Users\chidi\AppData\Local\Temp\ipykernel_5856\218216156.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t = torch.tensor(X[:, 1:2], dtype=torch.float32, requires_grad=True).to(device)
C:\Users\chidi\AppData\Local\Temp\ipykernel_5856\218216156.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.u = torch.tensor(u, dtype=torch.f

L-BFGS-B pretraining begins
STRidge begins


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn